In [ ]:
import pandas as pd
import json
import os
import numpy as np
import difflib
import re
import sys

sys.path.append('../..')

from tqdm import tqdm
from collections import deque, namedtuple

from IPython.display import display
from IPython.core.debugger import set_trace

# Listened log

In [ ]:
DATA = './source/music.json'

FIX_ARTISTS = {
    'Tarja Turunen': 'Tarja',
    'Thomas Bergersen': 'Two Steps from Hell',
    'Томас Бергерсен': 'Two Steps from Hell',
    'Two Steps from Hell & Thomas Bergersen': 'Two Steps from Hell',
    'Tribe': 'Amaranthe',
    'Two Steps From Hell': 'Two Steps from Hell',
    'Two Steps From Hell, Thomas Bergersen': 'Two Steps from Hell',
    'Two Steps From Hell, Nick Phoenix': 'Two Steps from Hell',
    'Андреас Вальдетофт': 'Andreas Waldetoft',
    'Andreas Waltedoft': 'Andreas Waldetoft',
    'Paradox Interactive': 'Andreas Waldetoft',
    'Meyer': 'Andreas Waldetoft',
    'Saxon': 'Sabaton',
    'Visions Of Atlantis': 'Visions of Atlantis',
    'Leaves´ Eyes': 'Leaves\' Eyes',
    'Leaves \' Eyes': 'Leaves\' Eyes',
    'Russell Allen & Anette Olzon': 'Allen/Olzon'
}

FIX_TITLES = {
    'inf': 'infinity'
}

def fix_artist(artist):
    return FIX_ARTISTS.get(artist, artist)

def fix_title(title):
    title = title.lower()
    if not re.match('.*(orchestral|instrumental).*', title):
        title = re.sub(r" ?\([^)]+\)", "", title)
    title = re.sub(r"listened to", "", title)
    title = re.sub(r"(:|~|/).*", "", title)
    title = title.strip()
    return FIX_TITLES.get(title, title)

def check_title(title):
    return "skipped" not in title and "google play music" not in title

with open(DATA, 'r') as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df['artist'] = df['description'].apply(fix_artist)
df['title'] = df['title'].apply(fix_title)
df = df[[check_title(t) for t in df.title]]
df = df.reset_index(drop=True)
df = df[['artist', 'title', 'time']]
df

# Tracks data

In [ ]:
from sqrt_data.api import DBConn
from models import Base, MpdSong, SongListened

DBConn()

In [ ]:
data = deque()

with DBConn.get_session() as db:
    for track in db.query(MpdSong).all():
        data.append({
            'title': track.title,
            'id': track.id,
            'artist': track.album_artist,
            'album': track.album
        })

df_t = pd.DataFrame(data)
df_t.title = df_t.title.apply(fix_title)
df_t.artist = df_t.artist.apply(fix_artist)
df_t = df_t.set_index('id')
df_t.head()

# Join listened with tracks

In [ ]:
tracks_by_name = {}

for track in df_t.itertuples(index=True):
    try:
        tracks_by_name[track.title].append(track)
    except KeyError:
        tracks_by_name[track.title] = [track]

track_names = list(tracks_by_name.keys())

In [ ]:
def get_tracks(title, cutoff=0.6):
    try:
        track = tracks_by_name[title]
        return track
    except KeyError:
        pass
    if len(title) > 5:
        try:
            matches = [t for t in track_names if title in t or t in title]
            if len(matches) > 1:
                match = difflib.get_close_matches(title, matches, cutoff=0.4)[0]
            else:
                match = matches[0]
            return tracks_by_name[match]
        except (StopIteration, IndexError):
            pass
    elif "Instrumental" in title:
        title = re.sub('Instrumental', "", title)
        title = re.sub('-', "", title)
        title = title.strip()
        return get_tracks(title, cutoff)
    matches = difflib.get_close_matches(title, track_names, cutoff=cutoff)
    try:
        closest = matches[0]
        return tracks_by_name[closest]
    except IndexError:
        return get_tracks(title, cutoff=0.3)
    return None

def pick_track(tracks, track_listened, id_series):
    same_artists = [t for t in tracks if t.artist == track_listened.artist]
    if len(same_artists) == 0:
        same_artists = [t for t in tracks if t.artist == 'nan']
        if len(same_artists) == 0:
            return None
    if len(same_artists) == 1:
        return same_artists[0]
    by_album = {t.album: t for t in same_artists}
    try:
        previous_track = df_t.loc[id_series[len(id_series) - 1]]
        if previous_track.artist == track_listened.artist:
            return by_album[previous_track.album]
    except (KeyError, TypeError):
        pass
    return same_artists[0]

id_series = deque()
not_matched = []
NotMatched = namedtuple('NotMatched', ['track', 'candidates'])

for track_listened in tqdm(list(df.itertuples(index=True))):
    tracks = get_tracks(track_listened.title)
    if tracks is None:
        id_series.append(None)
        not_matched.append(NotMatched(track_listened))
        continue
    track = pick_track(tracks, track_listened, id_series)
    if track is None:
        id_series.append(None)
        not_matched.append(NotMatched(track_listened, tracks))
        continue
    id_series.append(track.Index)

In [ ]:
df['id'] = id_series
df_nm = pd.DataFrame([t.track for t in not_matched])
# display(len(id_series))

with pd.option_context('display.max_rows', 1000):
    display(df_nm)

In [ ]:
data = list(df.itertuples(index=False))
with DBConn.get_session() as db:
    for record in tqdm(data):
        if record.id is not None:
            listened = SongListened(song_id=record.id, time=record.time)
            db.merge(listened)
    db.commit()